In [3]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
import pickle
import pandas as pd 
import numpy as np
from keras import backend as k
import time 

In [4]:
#load model
model = load_model("./models/vae.h5")
# summarize model.
model.summary()
with open("./models/LabelEncoders_dic.pickle","rb") as f:
    encoder_dic=pickle.load(f)
with open("./models/MinMaxScalers_dic.pickle","rb") as f:
    scaler_dic=pickle.load(f)

NameError: name 'k' is not defined

In [3]:
def transformer_df(df):
    for c in df.columns :
        if (df[c].dtype =="object"):
            encoder = encoder_dic[c]
            df[c]=encoder.transform(df[c])
    return df

In [77]:
def normaliser_all_columns(df):
    for c in df.columns :
        df[c]=scaler_dic[c].transform(df[c].values.reshape(-1,1))
    return df

In [78]:
df =pd.read_csv("../../data/full_data_small_datased.csv")

In [88]:
test_df = pd.DataFrame()
test_df = test_df.append(df.iloc[0])
test_df.head()

,Class,HH_L0.01_covariance,HH_L0.01_magnitude,HH_L0.01_mean,HH_L0.01_pcc,HH_L0.01_radius,HH_L0.01_std,HH_L0.01_weight,HH_L0.1_covariance,HH_L0.1_magnitude,...,MI_dir_L0.1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L1_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L3_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L5_weight
0,0.0,0.0,60.0,60.0,0.0,0.0,0.0,1.0,0.0,60.0,...,1.0,60.0,0.0,1.0,60.0,0.0,1.0,60.0,0.0,1.0


In [89]:
target = test_df.Class

In [90]:
x_test = test_df.drop(['Class'],axis=1)

In [91]:
x_test=transformer_df(x_test)
x_test=normaliser_all_columns(x_test)

In [92]:
y_pred=model.predict([x_test]) 

In [95]:
def prediction(xtest):
    threshold=2.8 
    start_time = time.time() 
    y_pred=model.predict(xtest) 
    print("--- %s seconds ---" % (time.time() - start_time))    
    y_dist=np.linalg.norm(xtest-y_pred,axis=-1) 
    z=zip(y_dist >= threshold , y_dist) 
    y_label=[]
    error=[]
    for idx ,(is_anomaly , y_dist) in enumerate(z):
        if is_anomaly : # if the value is True : which mean the distance is sup then the threshold 
            y_label.append(1)
        else:
            y_label.append(0)
        error.append(y_dist)
    return y_label

In [96]:
y_pred=prediction(x_test)

--- 0.006682872772216797 seconds ---


In [99]:
y_pred[0]

0